# PASO 2023  13-ago-2023
## Recuento Provisorio Hurlingham

453 mesas argentinos
28 mesas extranjeros

### Total de mesas escrutadas
463 de 481 (96.25%)

Circuito Electoral
00668 - Hurlingham
0668A - Morris
0668B - Tesei

In [1]:
import pandas as pd

In [ ]:
# Generate Mesa's Numbers

mesas = []

# argentinos
for i in range(1, 454):
  mesas.append('02134' + str(i).zfill(5) + 'X/10')

# extranjeros
for i in range(1, 29):
  mesas.append('02134090' + str(i).zfill(2) + 'E/10')

In [ ]:
# Make function to take all relevant values from a MESA and return df

def valores_mesa(URI):
    global totales, columns
    payload = ""
    response = requests.request("GET", URI, data=payload)

    data = response.json()
    # Generate specifc data

    escuela = data['fathers'][0]['name']
    circuito = data['fathers'][1]['name']
    mesa = data['id']['idAmbito']['codigo']
    
    cols = ['nombre', 'votos', 'perc', 'percAbs']

    for partido in data['partidos']:
        for lista in partido['listas']:
            df = pd.DataFrame.from_dict([lista])
            df['escuela'] = escuela
            df['circuito'] = circuito
            df['mesa'] = mesa
            totales = pd.concat([totales, df[columns]])
    print(response, 'mesa ' + mesa)

In [ ]:
# Request data

URI = "https://resultados.gob.ar/backend-difu/scope/data/getScopeData/"
columns = ['circuito', 'escuela', 'mesa', 'nombre', 'votos', 'perc', 'percAbs']
totales = pd.DataFrame(columns=columns)

for i in mesas:
    valores_mesa(URI+i)
    
totales.reset_index(inplace = True, drop=True)
totales.drop_duplicates(inplace=True) # por las dudas!
totales = totales[columns]

In [ ]:
# Save to .csv
totales.to_csv('PASO2023_escrutinio_provisorio_por_mesa_hurlingham.csv', index=False)

In [2]:
# Load from .csv
totales = pd.read_csv('PASO2023_escrutinio_provisorio_por_mesa_hurlingham.csv')

In [3]:
totales

,circuito,escuela,mesa,nombre,votos,perc,percAbs
0,00668,ESCUELA EP N°9/ ES N°17,0213400001X,CELESTE Y BLANCA - 1,50.0,51.02,22.12
1,00668,ESCUELA EP N°9/ ES N°17,0213400001X,CELESTE Y BLANCA - 2,48.0,48.97,21.23
2,00668,ESCUELA EP N°9/ ES N°17,0213400002X,CELESTE Y BLANCA - 1,59.0,53.63,26.94
3,00668,ESCUELA EP N°9/ ES N°17,0213400002X,CELESTE Y BLANCA - 2,51.0,46.36,23.28
4,00668,ESCUELA EP N°9/ ES N°17,0213400003X,CELESTE Y BLANCA - 2,58.0,53.21,25.00
...,...,...,...,...,...,...,...
9615,0668A,JARDIN DE INF.MUNICIPAL N°4,0213409028E,IZQUIERDA ANTICAPITALISTA,0.0,0.00,0.00
9616,0668A,JARDIN DE INF.MUNICIPAL N°4,0213409028E,A- AZUL Y ROJO,0.0,0.00,0.00
9617,0668A,JARDIN DE INF.MUNICIPAL N°4,0213409028E,A- UNIDAD OBRERA,0.0,0.00,0.00
9618,0668A,JARDIN DE INF.MUNICIPAL N°4,0213409028E,"1A- TIERRA, TECHO Y TRABAJO",0.0,0.00,0.00


In [4]:
# Check against official report 
# https://resultados.gob.ar/elecciones/10/81/1/-1/-1/Buenos-Aires/Secci%C3%B3n-Primera/Hurlingham

# OK!

totales.groupby(['nombre']).agg({'votos': 'sum'})

,votos
nombre,
"1A- TIERRA, TECHO Y TRABAJO",611.0
A- AZUL Y ROJO,468.0
A- UNIDAD OBRERA,333.0
A- UNIR Y FORTALECER LA IZQUIERDA,1941.0
ACCION Y SOLIDARIDAD,10.0
B- UNIDAD DE LUCHADORES Y LA IZQUIERDA,911.0
C- ANTICORRUPCIÓN,4.0
CELESTE,262.0
CELESTE Y BLANCA - 1,56264.0


# Análisis

In [5]:
# Clean Data

# Votos > 0
clean_data = totales[totales['votos'] > 0].copy()


# Nombres
names = ['CELESTE Y BLANCA - 1', 'CELESTE Y BLANCA - 2',
       'LA FUERZA DEL CAMBIO', 'FALTA MENOS PARA VIVIR SIN MIEDO',
       'LIBERTAD POR SIEMPRE']

mask = clean_data['nombre'].isin(names)
clean_data['nombres'] = clean_data['nombre'].where(mask, 'OTROS', inplace=True)

# PCT total and by circuit
clean_data['percAbs_valid'] = clean_data['votos'] / clean_data['votos'].sum()

votos_circuito = clean_data.groupby('circuito').agg(votos_circuito=('votos', 'sum'))
clean_data = pd.merge(clean_data, votos_circuito, on="circuito")
clean_data['percAbs_circuit'] = clean_data['votos'] / clean_data['votos_circuito']  


In [6]:
# Listas por Circuito Electoral

clean_data.groupby(['circuito', 'nombre']).agg({'votos': 'sum', 'percAbs_circuit': 'sum'})

votos  percAbs_circuit
circuito nombre                                                    
00668    CELESTE Y BLANCA - 1              18294.0         0.328686
         CELESTE Y BLANCA - 2              17562.0         0.315534
         FALTA MENOS PARA VIVIR SIN MIEDO   6438.0         0.115671
         LA FUERZA DEL CAMBIO               5518.0         0.099141
         LIBERTAD POR SIEMPRE               6059.0         0.108861
         OTROS                              1787.0         0.032107
0668A    CELESTE Y BLANCA - 1              17642.0         0.398734
         CELESTE Y BLANCA - 2              15730.0         0.355520
         FALTA MENOS PARA VIVIR SIN MIEDO   3020.0         0.068256
         LA FUERZA DEL CAMBIO               2158.0         0.048774
         LIBERTAD POR SIEMPRE               4367.0         0.098700
         OTROS                              1328.0         0.030015
0668B    CELESTE Y BLANCA - 1              20328.0         0.368896
         CELESTE Y BLANCA - 2              17598.0         0.319354
         FALTA MENOS PARA VIVIR SIN MIEDO   4664.0         0.084638
         LA FUERZA DEL CAMBIO               3800.0         0.068959
         LIBERTAD POR SIEMPRE               6722.0         0.121985
         OTROS                              1993.0         0.036167

In [7]:
# Mesas Extranjeros

clean_data['mesa_ext'] = clean_data['mesa'].str.strip().str[-1] == 'E'

results = pd.merge(
clean_data.groupby(['mesa_ext', 'nombre']).agg(votos=('votos', 'sum')).reset_index(),
clean_data.groupby(['mesa_ext']).agg(votos_total_mesa_tipo=('votos', 'sum')).reset_index(),
on="mesa_ext")

results['pct'] = results['votos'] / results['votos_total_mesa_tipo']
results

,mesa_ext,nombre,votos,votos_total_mesa_tipo,pct
0,False,CELESTE Y BLANCA - 1,54364.0,150869.0,0.360339
1,False,CELESTE Y BLANCA - 2,49418.0,150869.0,0.327556
2,False,FALTA MENOS PARA VIVIR SIN MIEDO,13762.0,150869.0,0.091218
3,False,LA FUERZA DEL CAMBIO,11348.0,150869.0,0.075218
4,False,LIBERTAD POR SIEMPRE,16934.0,150869.0,0.112243
5,False,OTROS,5043.0,150869.0,0.033426
6,True,CELESTE Y BLANCA - 1,1900.0,4139.0,0.459048
7,True,CELESTE Y BLANCA - 2,1472.0,4139.0,0.355641
8,True,FALTA MENOS PARA VIVIR SIN MIEDO,360.0,4139.0,0.086978
9,True,LA FUERZA DEL CAMBIO,128.0,4139.0,0.030925
